In [5]:
import pandas as pd

In [19]:
claims_data = pd.read_csv('claims_data.csv')

cust_data = pd.read_csv('cust_data.csv')

In [ ]:
1. Import claims_data.csv and cust_data.csv which is provided to you and combine the two datasets appropriately to create a 360-degree view of 
the data. Use the same for the subsequent questions.

In [ ]:
import pandas as pd

claims_data = pd.read_csv('claims_data.csv')
cust_data = pd.read_csv('cust_data.csv')

merged_data = pd.merge(claims_data, cust_data, left_on='customer_id', right_on='cust_id', how='left')

print(merged_data.head())


In [ ]:
2. Perform a data audit for the datatypes and find out if there are any mismatch within the current datatypes of the columns and their 
business significance.

In [ ]:
import pandas as pd

claims_data = pd.read_csv('claims_data.csv')
cust_data = pd.read_csv('cust_data.csv')

claims_data_dtypes = claims_data.dtypes
cust_data_dtypes = cust_data.dtypes

expected_claims_data_dtypes = {
    'claim_id': 'int64',
    'customer_id': 'int64',
    'incident_cause': 'object',
    'claim_date': 'datetime64',
    'claim_area': 'object',
    'police_report': 'object',
    'claim_type': 'object',
    'claim_amount': 'float64',
    'total_policy_claims': 'int64',
    'fraudulent': 'bool'
}

expected_cust_data_dtypes = {
    'cust_id': 'int64',
    'gender': 'object',
    'dateofbirth': 'datetime64',
    'state': 'object',
    'contact': 'object',
    'segment': 'object'
}

claims_data_mismatches = claims_data_dtypes != pd.Series(expected_claims_data_dtypes)
cust_data_mismatches = cust_data_dtypes != pd.Series(expected_cust_data_dtypes)

print("Claims Data Type Mismatches:")
print(claims_data_mismatches[claims_data_mismatches].index.tolist())

print("\nCustomer Data Type Mismatches:")
print(cust_data_mismatches[cust_data_mismatches].index.tolist())

print("\nActual Data Types in Claims Data:")
print(claims_data_dtypes)

print("\nActual Data Types in Customer Data:")
print(cust_data_dtypes)


In [ ]:
3. Convert the column claim_amount to numeric. Use the appropriate modules/attributes to remove the $ sign.

In [ ]:
import pandas as pd

claims_data = pd.read_csv('claims_data.csv')

claims_data['claim_amount'] = claims_data['claim_amount'].replace({'\$': ''}, regex=True)

claims_data['claim_amount'] = pd.to_numeric(claims_data['claim_amount'], errors='coerce')

print(claims_data[['claim_amount']].head())


In [ ]:
4. Of all the injury claims, some of them have gone unreported with the police. Create an alert flag (1,0) for all such claims.

In [ ]:
import pandas as pd

claims_data = pd.read_csv('claims_data.csv')

injury_claims = claims_data[claims_data['incident_cause'] == 'injury']

injury_claims['alert_flag'] = injury_claims['police_report'].apply(lambda x: 1 if x in ['No', 'Unknown'] else 0)

claims_data.loc[claims_data['incident_cause'] == 'injury', 'alert_flag'] = injury_claims['alert_flag']

print(claims_data[['claim_id', 'incident_cause', 'police_report', 'alert_flag']].head())


In [ ]:
5. One customer can claim for insurance more than once and in each claim, multiple categories of claims can be involved. However, customer ID 
should remain unique.

In [ ]:
import pandas as pd

claims_data = pd.read_csv('claims_data.csv')
customer_demographics = pd.read_csv('customer_demographics.csv')

claims_data = claims_data.rename(columns={'customer_id': 'cust_id'})
merged_data = pd.merge(claims_data, customer_demographics, on='cust_id', how='left')

merged_data['claim_date'] = pd.to_datetime(merged_data['claim_date'])

merged_data_sorted = merged_data.sort_values(by='claim_date', ascending=False)

merged_data_unique = merged_data_sorted.drop_duplicates(subset='cust_id', keep='first')

print(merged_data_unique[['cust_id', 'claim_id', 'incident_cause', 'claim_date', 'claim_amount']].head())


In [ ]:
6. Check for missing values and impute the missing values with an appropriate value. (mean for continuous and mode for categorical).

In [ ]:
import pandas as pd

claims_data = pd.read_csv('claims_data.csv')
customer_demographics = pd.read_csv('customer_demographics.csv')

claims_data = claims_data.rename(columns={'customer_id': 'cust_id'})
merged_data = pd.merge(claims_data, customer_demographics, on='cust_id', how='left')

missing_values = merged_data.isnull().sum()
print("Missing values per column:")
print(missing_values)

continuous_columns = merged_data.select_dtypes(include=['float64', 'int64']).columns
merged_data[continuous_columns] = merged_data[continuous_columns].fillna(merged_data[continuous_columns].mean())

categorical_columns = merged_data.select_dtypes(include=['object']).columns
for col in categorical_columns:
    merged_data[col] = merged_data[col].fillna(merged_data[col].mode()[0])

missing_values_after_imputation = merged_data.isnull().sum()
print("\nMissing values after imputation:")
print(missing_values_after_imputation)

print("\nData after imputation (first 5 rows):")
print(merged_data.head())


In [ ]:
7. Calculate the age of customers in years. Based on the age, categorize the customers according to the below criteria.
Children < 18
Youth 18-30
Adult 30-60
Senior > 60

In [ ]:
import pandas as pd

claims_data = pd.read_csv('claims_data.csv')
customer_demographics = pd.read_csv('customer_demographics.csv')

claims_data = claims_data.rename(columns={'customer_id': 'cust_id'})
merged_data = pd.merge(claims_data, customer_demographics, on='cust_id', how='left')

merged_data['dateofbirth'] = pd.to_datetime(merged_data['dateofbirth'])
merged_data['age'] = (pd.to_datetime('today') - merged_data['dateofbirth']).dt.days // 365

def categorize_age(age):
    if age < 18:
        return 'Children'
    elif 18 <= age <= 30:
        return 'Youth'
    elif 30 < age <= 60:
        return 'Adult'
    else:
        return 'Senior'

merged_data['age_category'] = merged_data['age'].apply(categorize_age)

print(merged_data[['cust_id', 'age', 'age_category']].head())


In [ ]:
8. What is the average amount claimed by the customers from various segments?

In [ ]:
import pandas as pd

claims_data = pd.read_csv('claims_data.csv')
customer_demographics = pd.read_csv('customer_demographics.csv')

claims_data = claims_data.rename(columns={'customer_id': 'cust_id'})
merged_data = pd.merge(claims_data, customer_demographics, on='cust_id', how='left')

average_claim_by_segment = merged_data.groupby('segment')['claim_amount'].mean().reset_index()

print(average_claim_by_segment)


In [ ]:
9. What is the total claim amount based on incident cause for all the claims that have been done at least 20 days prior to 1st of October, 2018.

In [ ]:
import pandas as pd

claims_data = pd.read_csv('claims_data.csv')

claims_data['claim_date'] = pd.to_datetime(claims_data['claim_date'])

cutoff_date = pd.to_datetime('2018-10-01') - pd.Timedelta(days=20)

filtered_claims = claims_data[claims_data['claim_date'] <= cutoff_date]

total_claim_by_cause = filtered_claims.groupby('incident_cause')['claim_amount'].sum().reset_index()

print(total_claim_by_cause)


In [ ]:
10. How many adults from TX, DE and AK claimed insurance for driver related issues and causes?

In [ ]:
import pandas as pd

claims_data = pd.read_csv('claims_data.csv')
customer_demographics = pd.read_csv('customer_demographics.csv')

customer_demographics['dateofbirth'] = pd.to_datetime(customer_demographics['dateofbirth'])
customer_demographics['age'] = (pd.to_datetime('today') - customer_demographics['dateofbirth']).dt.days // 365

adults = customer_demographics[(customer_demographics['age'] >= 30) & (customer_demographics['age'] <= 60)]

merged_data = pd.merge(claims_data, adults[['cust_id', 'state']], left_on='customer_id', right_on='cust_id', how='inner')

filtered_data = merged_data[(merged_data['state'].isin(['TX', 'DE', 'AK'])) &
                             (merged_data['incident_cause'].str.contains('driver', case=False, na=False))]

unique_customers = filtered_data['customer_id'].nunique()

print(f"Number of adults from TX, DE, and AK who claimed insurance for driver-related issues: {unique_customers}")


In [ ]:
11. Draw a pie chart between the aggregated value of claim amount based on gender and segment. Represent the claim amount as a percentage on
the pie chart.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

claims_data = pd.read_csv('claims_data.csv')
customer_demographics = pd.read_csv('customer_demographics.csv')

merged_data = pd.merge(claims_data, customer_demographics[['cust_id', 'gender', 'segment']], 
                       left_on='customer_id', right_on='cust_id', how='inner')

grouped_data = merged_data.groupby(['gender', 'segment'])['claim_amount'].sum().reset_index()

grouped_data['label'] = grouped_data['gender'] + " - " + grouped_data['segment']

plt.figure(figsize=(8, 8))
plt.pie(grouped_data['claim_amount'], labels=grouped_data['label'], autopct='%1.1f%%', startangle=90)
plt.title('Claim Amount by Gender and Segment')
plt.axis('equal')
plt.show()


In [ ]:
12. Among males and females, which gender had claimed the most for any type of driver related issues? E.g. This metric can be compared using a 
bar chart.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

claims_data = pd.read_csv('claims_data.csv')
customer_demographics = pd.read_csv('customer_demographics.csv')

merged_data = pd.merge(claims_data, customer_demographics[['cust_id', 'gender']], 
                       left_on='customer_id', right_on='cust_id', how='inner')

driver_claims = merged_data[merged_data['incident_cause'].str.contains('driver', case=False, na=False)]

gender_claims = driver_claims.groupby('gender')['claim_amount'].sum().reset_index()

plt.figure(figsize=(8, 6))
plt.bar(gender_claims['gender'], gender_claims['claim_amount'], color=['blue', 'pink'])
plt.title('Total Claim Amount for Driver-Related Issues by Gender')
plt.xlabel('Gender')
plt.ylabel('Total Claim Amount')
plt.show()


In [ ]:
13. Which age group had the maximum fraudulent policy claims? Visualize it on a bar chart.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

claims_data = pd.read_csv('claims_data.csv')
customer_demographics = pd.read_csv('customer_demographics.csv')

merged_data = pd.merge(claims_data, customer_demographics[['cust_id', 'gender', 'dateofbirth']], 
                       left_on='customer_id', right_on='cust_id', how='inner')

merged_data['dateofbirth'] = pd.to_datetime(merged_data['dateofbirth'])
merged_data['age'] = (pd.to_datetime('today') - merged_data['dateofbirth']).dt.days // 365

def age_group(age):
    if age < 18:
        return 'Children'
    elif 18 <= age <= 30:
        return 'Youth'
    elif 30 < age <= 60:
        return 'Adult'
    else:
        return 'Senior'

merged_data['age_group'] = merged_data['age'].apply(age_group)

fraudulent_claims = merged_data[merged_data['fraudulent'] == 1]
age_group_fraud = fraudulent_claims.groupby('age_group').size().reset_index(name='fraudulent_claim_count')

plt.figure(figsize=(8, 6))
plt.bar(age_group_fraud['age_group'], age_group_fraud['fraudulent_claim_count'], color='orange')
plt.title('Number of Fraudulent Claims by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Number of Fraudulent Claims')
plt.xticks(rotation=45)
plt.show()


In [ ]:
14. Visualize the monthly trend of the total amount that has been claimed by the customers. Ensure that on the “month” axis, the month is in a 
chronological order not alphabetical order.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

claims_data = pd.read_csv('claims_data.csv')

claims_data['claim_date'] = pd.to_datetime(claims_data['claim_date'])

claims_data['month_year'] = claims_data['claim_date'].dt.to_period('M')

monthly_claims = claims_data.groupby('month_year')['claim_amount'].sum().reset_index()

plt.figure(figsize=(10, 6))
plt.plot(monthly_claims['month_year'].astype(str), monthly_claims['claim_amount'], marker='o', color='b')

plt.title('Monthly Trend of Total Claim Amount')
plt.xlabel('Month-Year')
plt.ylabel('Total Claim Amount')
plt.xticks(rotation=45, ha='right')
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
15. What is the average claim amount for gender and age categories and suitably represent the above using a facetted bar chart, one facet that 
represents fraudulent claims and the other for non-fraudulent claims.

Based on the conclusions from exploratory analysis as well as suitable statistical tests, answer the below questions. Please include a detailed 
write-up on the parameters taken into consideration, the Hypothesis testing steps, conclusion from the p-values and the business implications of 
the statements.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

claims_data = pd.read_csv('claims_data.csv')
customer_data = pd.read_csv('customer_demographics.csv')

claims_data['claim_date'] = pd.to_datetime(claims_data['claim_date'])

merged_data = pd.merge(claims_data, customer_data, left_on='customer_id', right_on='cust_id', how='inner')

def categorize_age(age):
    if age < 18:
        return 'Children'
    elif 18 <= age <= 30:
        return 'Youth'
    elif 30 < age <= 60:
        return 'Adult'
    else:
        return 'Senior'

merged_data['age'] = pd.to_datetime('today').year - pd.to_datetime(merged_data['dateofbirth']).dt.year

merged_data['age_group'] = merged_data['age'].apply(categorize_age)

avg_claims = merged_data.groupby(['gender', 'age_group', 'fraudulent'])['claim_amount'].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.set(style="whitegrid")
g = sns.catplot(x='age_group', y='claim_amount', hue='gender', col='fraudulent', 
                data=avg_claims, kind='bar', height=5, aspect=1.5)

g.set_axis_labels('Age Group', 'Average Claim Amount')
g.set_titles('Fraudulent Claims: {col_name}')
g.fig.suptitle('Average Claim Amount by Gender and Age Group for Fraudulent vs Non-Fraudulent Claims', fontsize=16)
g.fig.tight_layout()
g.fig.subplots_adjust(top=0.85)

plt.show()

from scipy.stats import ttest_ind

fraudulent_claims = merged_data[merged_data['fraudulent'] == 1]['claim_amount']
non_fraudulent_claims = merged_data[merged_data['fraudulent'] == 0]['claim_amount']

t_stat, p_value = ttest_ind(fraudulent_claims, non_fraudulent_claims, equal_var=False)

print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")

if p_value < 0.05:
    print("Reject the null hypothesis. There is a significant difference in the claim amounts between fraudulent and non-fraudulent claims.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference in the claim amounts between fraudulent and non-fraudulent claims.")


In [ ]:
16. Is there any similarity in the amount claimed by males and females?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind

claims_data = pd.read_csv('claims_data.csv')
customer_data = pd.read_csv('customer_demographics.csv')

merged_data = pd.merge(claims_data, customer_data, left_on='customer_id', right_on='cust_id', how='inner')

merged_data['claim_amount'] = pd.to_numeric(merged_data['claim_amount'], errors='coerce')

plt.figure(figsize=(10, 6))
sns.boxplot(x='gender', y='claim_amount', data=merged_data)
plt.title('Claim Amount Distribution by Gender')
plt.xlabel('Gender')
plt.ylabel('Claim Amount')
plt.show()

male_claims = merged_data[merged_data['gender'] == 'Male']['claim_amount']
female_claims = merged_data[merged_data['gender'] == 'Female']['claim_amount']

t_stat, p_value = ttest_ind(male_claims.dropna(), female_claims.dropna(), equal_var=False)

print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")

if p_value < 0.05:
    print("Reject the null hypothesis. There is a significant difference in the average claim amounts between males and females.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference in the average claim amounts between males and females.")


In [ ]:
17. Is there any relationship between age category and segment?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

claims_data = pd.read_csv('claims_data.csv')
customer_data = pd.read_csv('customer_demographics.csv')

merged_data = pd.merge(claims_data, customer_data, left_on='customer_id', right_on='cust_id', how='inner')

merged_data['dateofbirth'] = pd.to_datetime(merged_data['dateofbirth'], errors='coerce')

current_date = pd.to_datetime('today')
merged_data['age'] = (current_date - merged_data['dateofbirth']).dt.days // 365

def categorize_age(age):
    if age < 18:
        return 'Children'
    elif 18 <= age <= 30:
        return 'Youth'
    elif 30 < age <= 60:
        return 'Adult'
    else:
        return 'Senior'

merged_data['age_category'] = merged_data['age'].apply(categorize_age)

age_segment_crosstab = pd.crosstab(merged_data['age_category'], merged_data['segment'], margins=True, margins_name="Total")

plt.figure(figsize=(10, 6))
age_segment_crosstab.drop(columns='All').plot(kind='bar', stacked=True, colormap='Set2')
plt.title('Relationship between Age Category and Segment')
plt.xlabel('Age Category')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.legend(title="Segment", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

chi2_stat, p_val, dof, expected = chi2_contingency(age_segment_crosstab.drop(columns='All'))

print(f"Chi-Square Statistic: {chi2_stat}")
print(f"P-Value: {p_val}")

if p_val < 0.05:
    print("Reject the null hypothesis. There is a significant relationship between age category and segment.")
else:
    print("Fail to reject the null hypothesis. There is no significant relationship between age category and segment.")


In [ ]:
18. The current year has shown a significant rise in claim amounts as compared to 2016-17 fiscal average which was $10,000.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

claims_data = pd.read_csv('claims_data.csv')

claims_data['claim_date'] = pd.to_datetime(claims_data['claim_date'])

current_year = pd.to_datetime('today').year
claims_data['year'] = claims_data['claim_date'].dt.year

current_year_claims = claims_data[claims_data['year'] == current_year]

average_claim_current_year = current_year_claims['claim_amount'].mean()

fiscal_year_2016_17_average = 10000

print(f"Average claim amount for the current year ({current_year}): ${average_claim_current_year:.2f}")
print(f"Fiscal year 2016-17 average claim amount: ${fiscal_year_2016_17_average}")

if average_claim_current_year > fiscal_year_2016_17_average:
    print("There is a significant rise in claim amounts compared to the fiscal year 2016-17.")
else:
    print("Claim amounts have not risen significantly compared to the fiscal year 2016-17.")

labels = ['Current Year Average', '2016-17 Fiscal Average']
values = [average_claim_current_year, fiscal_year_2016_17_average]

plt.figure(figsize=(8, 5))
plt.bar(labels, values, color=['skyblue', 'orange'])
plt.title(f"Comparison of Average Claim Amounts: {current_year} vs 2016-17 Fiscal Year")
plt.ylabel("Claim Amount ($)")
plt.show()


In [ ]:
19. Is there any difference between age groups and insurance claims?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime

claims_data = pd.read_csv('claims_data.csv')
customer_data = pd.read_csv('customer_demographics.csv')

customer_data['dateofbirth'] = pd.to_datetime(customer_data['dateofbirth'])

current_year = datetime.now().year
customer_data['age'] = current_year - customer_data['dateofbirth'].dt.year

def categorize_age(age):
    if age < 18:
        return 'Children'
    elif 18 <= age <= 30:
        return 'Youth'
    elif 30 < age <= 60:
        return 'Adult'
    else:
        return 'Senior'

customer_data['age_group'] = customer_data['age'].apply(categorize_age)

claims_data = claims_data.merge(customer_data[['cust_id', 'age_group']], left_on='customer_id', right_on='cust_id')

age_group_claims = claims_data.groupby('age_group')['claim_amount'].mean().reset_index()

age_groups = claims_data['age_group'].unique()
claims_by_age_group = [claims_data[claims_data['age_group'] == group]['claim_amount'] for group in age_groups]
anova_result = stats.f_oneway(*claims_by_age_group)

print(f'ANOVA Test Result: p-value = {anova_result.pvalue}')
if anova_result.pvalue < 0.05:
    print("There is a significant difference between age groups and insurance claims.")
else:
    print("There is no significant difference between age groups and insurance claims.")

plt.figure(figsize=(10, 6))
sns.boxplot(x='age_group', y='claim_amount', data=claims_data)
plt.title('Claim Amounts by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Claim Amount ($)')
plt.show()


In [ ]:
20. Is there any relationship between total number of policy claims and the claimed amount?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

claims_data = pd.read_csv('claims_data.csv')
customer_data = pd.read_csv('customer_demographics.csv')

merged_data = claims_data.merge(customer_data[['cust_id', 'gender', 'segment']], left_on='customer_id', right_on='cust_id')

plt.figure(figsize=(10, 6))
sns.scatterplot(x='total_policy_claims', y='claim_amount', data=merged_data)
plt.title('Relationship between Total Policy Claims and Claimed Amount')
plt.xlabel('Total Policy Claims')
plt.ylabel('Claim Amount ($)')
plt.show()

correlation = merged_data[['total_policy_claims', 'claim_amount']].corr().iloc[0, 1]
print(f"Pearson correlation coefficient: {correlation}")

if correlation > 0.7:
    print("There is a strong positive correlation between the total number of policy claims and the claimed amount.")
elif correlation > 0.3:
    print("There is a moderate positive correlation between the total number of policy claims and the claimed amount.")
else:
    print("There is little to no correlation between the total number of policy claims and the claimed amount.")
